In [1]:
import tensorflow as tf

import pandas as pd
import codecs
import numpy as np
import math
import csv
from sklearn.preprocessing import MinMaxScaler, StandardScaler
from sklearn.metrics import mean_squared_error
from sklearn.metrics import mean_absolute_error
from sklearn.metrics import recall_score, precision_score # New
from sklearn.metrics import f1_score
from sklearn.metrics import confusion_matrix
from sklearn import preprocessing
import matplotlib.pyplot as plt
import tensorflow.keras.preprocessing
#from tensorflow.keras.callbacks import ModelCheckpoint, CSVLogger, EarlyStopping, LearningRateScheduler, ReduceLROnPlateau
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.layers import Dense, Input, Flatten, Add, Multiply, Subtract
from tensorflow.keras import backend as K
from tensorflow.keras.layers import Activation, BatchNormalization
# regularizers
from tensorflow.keras.layers import Conv1D, Conv2D
from tensorflow.keras.layers import MaxPooling1D
from tensorflow.keras.layers import concatenate
from tensorflow.keras.layers import LSTM, Bidirectional
from tensorflow.keras.utils import plot_model
from tensorflow.keras.layers import Dropout, SpatialDropout1D
from scipy import signal
import openpyxl
import seaborn as sns
import matplotlib.pyplot as plt
import warnings
warnings.filterwarnings('ignore')

from matplotlib import pylab as plt
import matplotlib.dates as mdates
import seaborn as sns
import sklearn
from sklearn.model_selection import train_test_split
from numpy.linalg import svd
import gc
from keijzer import *
from tensorflow import keras
from keras.models import Model
from keras.layers import *
from tensorflow.keras.optimizers import Adam,SGD
from keras.layers import Dense, Input, Flatten, Add, concatenate, Dropout, Activation, Multiply, Embedding, Layer, Reshape
from keras.layers import LSTM, Bidirectional, GlobalMaxPooling1D, GlobalAveragePooling1D  
from keras.layers import Conv1D, AveragePooling1D, MaxPooling1D
#from keras.ops import convert_to_tensor, convert_to_numpy
#from keras.utils import plot_model
from keras import activations
from sklearn.pipeline import Pipeline
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import StratifiedKFold
from keras.callbacks import ModelCheckpoint, CSVLogger, EarlyStopping, LearningRateScheduler, ReduceLROnPlateau
import math

class LossHistory(tensorflow.keras.callbacks.Callback):  # history = LossHistory()
    def init(self):  # history.init()
        self.losses = []
        # self.accs = []
        self.val_losses = []
        # self.val_accs = []
        self.rmses = []
        self.mses = []
        self.maes = []
        self.mapes = []
        self.val_rmses = []
        self.val_mses = []
        self.val_maes = []
        self.val_mapes = []

    def on_epoch_end(self, batch, logs={}):
        self.losses.append(logs.get('loss'))
        # self.accs.append(logs.get('acc'))
        self.val_losses.append(logs.get('val_loss'))
        # self.val_accs.append(logs.get('val_accuracy
        self.rmses.append(logs.get('root_mean_squared_error'))
        self.mses.append(logs.get('mean_squared_error'))
        self.maes.append(logs.get('mean_absolute_error'))
        self.mapes.append(logs.get('mean_absolute_percentage_error'))
        self.val_rmses.append(logs.get('val_root_mean_squared_error'))
        self.val_mses.append(logs.get('val_mean_squared_error'))
        self.val_maes.append(logs.get('val_mean_absolute_error'))
        self.val_mapes.append(logs.get('val_mean_absolute_percentage_error'))



def root_squared_mean_error(y_true, y_pred):
    return K.mean((K.abs(y_pred - y_true))*K.square(y_true-K.mean(y_true)))*100

def mse_mae(y_true, y_pred):
    return K.mean(K.square(y_pred - y_true))+K.mean(K.abs(y_pred - y_true))

from keras.layers.advanced_activations import PReLU

def mean_absolute_percentage_error(y_true, y_pred):
    y_true, y_pred = np.array(y_true), np.array(y_pred)
    return np.mean(np.abs((y_true - y_pred) / y_true)) * 100

def root_mean_squared_error(y_true, y_pred):
        return K.sqrt(K.mean(K.square(y_pred - y_true))) 

In [2]:
import scipy as sc
import os

os.environ["CUDA_VISIBLE_DEVICES"] = '0' # 여러개 사용시 '0,1,2' 식으로 하나의 문자열에 입력
gpus = tf.config.experimental.list_physical_devices('GPU') # 호스트 러나임에 표시되는 GPU 장치 목록 반환

if gpus: # 반환된 GPU 장치 목록이 있다면
    try: # 해당 장치에 대한 메모리 증가 활성화 여부 설정
        tf.config.experimental.set_memory_growth(gpus[0], True)
    except RuntimeError as e: # try문 실패시에 에러문구 출력
        print(e)

print(tf.config.list_physical_devices('GPU'))

[PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]


In [3]:
data_path = 'C:/Users/smartgrid_AI/Desktop/windpower_ramprate/ercot_2020.csv'
data_source1 = pd.read_csv(data_path)
dat_source1 = pd.DataFrame(data_source1)
#dat_source1 = dat_source1.iloc[:-3,:]

data_path = 'C:/Users/smartgrid_AI/Desktop/windpower_ramprate/ercot_2021.csv'
data_source2 = pd.read_csv(data_path)
dat_source2 = pd.DataFrame(data_source2)
dat_source2 = dat_source2.iloc[:8761,:9]

data_path = 'C:/Users/smartgrid_AI/Desktop/windpower_ramprate/ercot_2022.csv'
data_source3 = pd.read_csv(data_path)
dat_source3 = pd.DataFrame(data_source3)
dat_source3 = dat_source3.iloc[:8760,:9]

data_path = 'C:/Users/smartgrid_AI/Desktop/windpower_ramprate/ercot_2023.csv'
data_source4 = pd.read_csv(data_path)
dat_source4 = pd.DataFrame(data_source4)
dat_source4 = dat_source4.iloc[:8760,:9]

dat_arr1 = np.array(dat_source1)
dat_arr2 = np.array(dat_source2)
dat_arr3 = np.array(dat_source3)
dat_arr4 = np.array(dat_source4)

dat_arr = np.concatenate([dat_arr1, dat_arr2, dat_arr3, dat_arr4],axis=0)
dat_arr.shape # 8784, 8761, 8760, 8760

dat_source = pd.DataFrame(dat_arr, columns=['Time-Date stamp','Date','ERCOT Load','Total Wind Output','Total Wind Installed','Wind Output, % of Load','Wind Output, % of Installed','1-hr MW change','1-hr % change'])
dat = pd.DataFrame()
dat['MW % change'] = np.array(dat_source.iloc[1:,3], dtype=float)-np.array(dat_source.iloc[0:-1,3], dtype=float)
dat['MW % change'] = np.divide(np.array(dat['MW % change']),np.array(dat_source.iloc[1:,4]))*100
dat['Total Wind Output'] = np.divide(np.array(dat_source.iloc[1:,3]), np.array(dat_source.iloc[1:,4]))

ramp_ratio = 20
load_ratio = 10
year_arr = np.concatenate([np.ones(8784)*2020, np.ones(8760)*2021, np.ones(8760)*2022, np.ones(8760)*2023])
year_df = pd.DataFrame(year_arr, columns=['Year'], dtype=int)
wind_df = pd.concat([year_df, dat], axis=1)
wind_df = wind_df.reset_index(drop=True)

std_scaler = sklearn.preprocessing.MinMaxScaler()
std_scaler_load = sklearn.preprocessing.MinMaxScaler()
std_scaler_ramp = sklearn.preprocessing.MinMaxScaler()

wind_df[['Total Wind Output']] = std_scaler.fit_transform(wind_df[['Total Wind Output']])
wind_df[['MW % change']] = std_scaler_ramp.fit_transform(wind_df[['MW % change']])
wind_df = wind_df.iloc[:-1,:]
ramp_df = pd.DataFrame(wind_df)
ramp_df.columns=['Year', 'Wind Change', 'Wind Output']
ramp_df

,Year,Wind Change,Wind Output
0,2020,0.620197,0.644724
1,2020,0.544580,0.658617
2,2020,0.570751,0.683924
3,2020,0.599594,0.721813
4,2020,0.495246,0.714187
...,...,...,...
35058,2023,0.774643,0.484984
35059,2023,0.735512,0.582157
35060,2023,0.601593,0.620917
35061,2023,0.536583,0.631322


In [4]:
def create_dataset(dataset, timesteps, output_timesteps, leadtime):
    dataX, dataY = [], []
    for i in range(len(dataset) - timesteps -output_timesteps - leadtime - 1):
        a = dataset[i:(i + timesteps), :]
        dataX.append(a)
        dataY.append(dataset[(i + timesteps + leadtime):(i+timesteps+output_timesteps+leadtime), :])
    return np.array(dataX), np.array(dataY)

def svdadd(X, start, end, b_size):
    dec_x = X[start:end, :, :]
    U, S, Vh = svd(dec_x.reshape(b_size, -1), full_matrices=True)

    high_sig = np.matmul(np.matmul(U[:,:high_ind], np.diag(S[0:high_ind])), Vh[:high_ind,:])
    low_sig = X[:dec_num, :, :].reshape(b_size,-1)-high_sig

    rec_x = np.zeros((b_size, b_size, high_ind))
    
    for i in range(high_ind):
        rec_x[:,:,i] = np.matmul((U[:,i]*S[i]).reshape(-1,1), Vh[i,:].reshape(1,-1))    
    return rec_x, low_sig

In [5]:
norm_df = np.array(wind_df.iloc[:,1]).reshape(-1,1)
timesteps = 24*7
output_timesteps = 24
leadtime = 6
num_features = 1
X, Y = create_dataset(norm_df, timesteps, output_timesteps, leadtime)
b_size = timesteps

In [6]:
dec_num = timesteps
high_ind = 35

addX = np.zeros((X.shape[0], dec_num, high_ind+1))

for k in range(X.shape[0]//dec_num):
    rec_x, low_sig = svdadd(X, k*dec_num, (k+1)*dec_num, b_size)
    addX[k*dec_num:k*dec_num+dec_num,:, :-1] = rec_x
    addX[k*dec_num:k*dec_num+dec_num,:, -1] = low_sig
    
rec_x, low_sig = svdadd(X, X.shape[0]-dec_num, X.shape[0], b_size)
addX[X.shape[0]-dec_num:X.shape[0], :, :-1] = rec_x
addX[X.shape[0]-dec_num:X.shape[0],:, -1] = low_sig

addX_det = addX[:, :, :-1]
addX_det = np.sum(addX_det, axis=2)
X = np.concatenate([X, addX], axis=2)
num_features = X.shape[2]

In [7]:
trvaX, teX, trvaY, teY = train_test_split(X, Y, test_size=0.1, shuffle=False)

idx = np.arange(trvaX.shape[0])
np.random.shuffle(idx)
strvaX = trvaX[idx]
strvaY = trvaY[idx]
strvaY = strvaY.reshape(-1, output_timesteps)

trvaY = trvaY.reshape(-1, output_timesteps)
teY = teY.reshape(-1, output_timesteps)

strX, svaX, strY, svaY = train_test_split(strvaX, strvaY, test_size=0.3, shuffle=False)
trX, vaX, trY, vaY = train_test_split(trvaX, trvaY, test_size=0.3, shuffle=False)

trvaX.shape, trvaY.shape, teX.shape, teY.shape

((31377, 168, 37), (31377, 24), (3487, 168, 37), (3487, 24))

In [8]:
def MAEMD(y_true, y_pred):
    return K.mean((K.abs(y_pred - y_true))*K.square(y_true-K.mean(y_true)))*100

## PatchMixer - With SVD With Shuffling

In [9]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
import numpy as np

# PatchMixerLayer definition
class PatchMixerLayer(keras.layers.Layer):
    def __init__(self, dim, a, kernel_size=8):
        super(PatchMixerLayer, self).__init__()
        self.resnet = keras.Sequential([
            layers.Conv1D(dim, kernel_size, groups=dim, padding="same"),
            layers.Activation("gelu"),
            layers.BatchNormalization()
        ])
        self.conv_1x1 = keras.Sequential([
            layers.Conv1D(a, 1),
            layers.Activation("gelu"),
            layers.BatchNormalization()
        ])
    
    def call(self, x):
        x = x + self.resnet(x)
        x = self.conv_1x1(x)
        return x

# RevIN implementation
class RevIN(layers.Layer):
    def __init__(self, num_features, eps=1e-5, affine=True, subtract_last=False):
        super(RevIN, self).__init__()
        self.num_features = num_features
        self.eps = eps
        self.affine = affine
        self.subtract_last = subtract_last
        if self.affine:
            self.affine_weight = self.add_weight(shape=(num_features,), initializer="ones", trainable=True)
            self.affine_bias = self.add_weight(shape=(num_features,), initializer="zeros", trainable=True)
    
    def call(self, x, mode):
        if mode == "norm":
            return self._normalize(x)
        elif mode == "denorm":
            return self._denormalize(x)
        else:
            raise NotImplementedError

    def _normalize(self, x):
        mean = tf.reduce_mean(x, axis=(1, 2), keepdims=True)
        stdev = tf.math.sqrt(tf.reduce_variance(x, axis=(1, 2), keepdims=True) + self.eps)
        if self.subtract_last:
            last = tf.expand_dims(x[:, -1, :], axis=1)
            x = (x - last) / stdev
        else:
            x = (x - mean) / stdev
        if self.affine:
            x = x * self.affine_weight + self.affine_bias
        return x

    def _denormalize(self, x):
        if self.affine:
            x = (x - self.affine_bias) / self.affine_weight
        stdev = tf.math.sqrt(tf.reduce_variance(x, axis=(1, 2), keepdims=True) + self.eps)
        mean = tf.reduce_mean(x, axis=(1, 2), keepdims=True)
        x = x * stdev + mean
        return x

# Main PatchMixer model definition
class PatchMixerModel(keras.Model):
    def __init__(self, input_shape, patch_size, stride, d_model, depth, kernel_size, forecasting, head_dropout):
        super(PatchMixerModel, self).__init__()
        self.lookback = input_shape[1]
        self.patch_size = patch_size
        self.stride = stride
        self.patch_num = (self.lookback - self.patch_size) // self.stride + 1
        self.d_model = d_model
        self.depth = depth
        self.kernel_size = kernel_size
        self.forecasting = forecasting
        self.head_dropout = head_dropout

        self.patch_mixer_blocks = [PatchMixerLayer(self.patch_num, self.patch_num, kernel_size) for _ in range(depth)]
        self.W_P = layers.Dense(d_model)
        
        self.head0 = keras.Sequential([
            layers.Flatten(),
            layers.Dense(forecasting),
            layers.Dropout(head_dropout)
        ])
        self.head1 = keras.Sequential([
            layers.Flatten(),
            layers.Dense(forecasting * 2, activation="gelu"),
            layers.Dropout(head_dropout),
            layers.Dense(forecasting),
            layers.Dropout(head_dropout)
        ])
    
    def call(self, x):
        batch_size = tf.shape(x)[0]
        x = tf.transpose(x, perm=[0, 2, 1])
        x_padded = tf.image.extract_patches(
            images=tf.expand_dims(x, axis=-1),
            sizes=[1, self.patch_size, 1, 1],
            strides=[1, self.stride, 1, 1],
            rates=[1, 1, 1, 1],
            padding="VALID"
        )
        #print(x_padded.shape)
        x = self.W_P(x_padded)
        x = tf.reshape(x, (batch_size, -1, self.d_model))
        u = self.head0(x)
        #print(x.shape, (self.lookback - self.patch_size),self.stride, self.patch_num)
        for block in self.patch_mixer_blocks:
            x = block(x)
        x = self.head1(x)
        return u + x

In [28]:
input_shape = (None, 168, 37)  # Example input shape

with tf.device('/gpu:0'):
    model = PatchMixerModel(
        input_shape=input_shape,
        patch_size=8,
        stride=5,
        d_model=33,
        depth=4,
        kernel_size=8,
        forecasting=24,
        head_dropout=0.1)
    model.build(input_shape=(168, 168, 37))
    model.summary()

Model: "patch_mixer_model_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
patch_mixer_layer_8 (PatchMi multiple                  1683      
_________________________________________________________________
patch_mixer_layer_9 (PatchMi multiple                  1683      
_________________________________________________________________
patch_mixer_layer_10 (PatchM multiple                  1683      
_________________________________________________________________
patch_mixer_layer_11 (PatchM multiple                  1683      
_________________________________________________________________
dense_8 (Dense)              multiple                  297       
_________________________________________________________________
sequential_28 (Sequential)   (168, 24)                 798360    
_________________________________________________________________
sequential_29 (Sequential)   (168, 24)         

In [29]:
gc.collect()

6246

In [30]:
    model.compile(loss=MAEMD, optimizer='adam', metrics=['mse','mae', MAEMD])
    early_stopping =EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)
    batch_size = 168
    epochs = 1000
    history = LossHistory()
    history.init()

In [31]:
    %%time
    b_size = 168
    hist = model.fit(strX, strY, epochs=epochs, batch_size=b_size, shuffle=False, validation_data=(svaX, svaY), callbacks=[history, early_stopping])  # , checkpoint

Epoch 1/1000
131/131 [==============================] - 27s 185ms/step - loss: 0.2283 - mse: 0.0949 - mae: 0.1889 - MAEMD: 0.2282 - val_loss: 0.1459 - val_mse: 0.0175 - val_mae: 0.1046 - val_MAEMD: 0.1470
Epoch 2/1000
131/131 [==============================] - 24s 182ms/step - loss: 0.1725 - mse: 0.0420 - mae: 0.1467 - MAEMD: 0.1724 - val_loss: 0.1496 - val_mse: 0.0196 - val_mae: 0.1115 - val_MAEMD: 0.1516
Epoch 3/1000
131/131 [==============================] - 24s 183ms/step - loss: 0.1610 - mse: 0.0375 - mae: 0.1401 - MAEMD: 0.1610 - val_loss: 0.1528 - val_mse: 0.0210 - val_mae: 0.1154 - val_MAEMD: 0.1543
Epoch 4/1000
131/131 [==============================] - 24s 183ms/step - loss: 0.1511 - mse: 0.0344 - mae: 0.1341 - MAEMD: 0.1511 - val_loss: 0.1393 - val_mse: 0.0167 - val_mae: 0.1016 - val_MAEMD: 0.1408
Epoch 5/1000
131/131 [==============================] - 24s 181ms/step - loss: 0.1454 - mse: 0.0327 - mae: 0.1316 - MAEMD: 0.1453 - val_loss: 0.1389 - val_mse: 0.0167 - val_mae: 0.

In [32]:
gc.collect()

189554

In [33]:
#model3.save('SCINet', save_format='tf')
testPredict = model.predict(teX, batch_size=b_size)

In [34]:
def npMAEMS(y_true, y_pred):
    return np.mean((abs(y_pred - y_true))*np.square(y_true))*100
def npMAEMD(y_true, y_pred):
    return np.mean((abs(y_pred - y_true))*np.square(y_true-np.mean(y_true)))*100
def npMSE(y_true, y_pred):
    return np.mean(np.square(-y_true+y_pred))
def npMAE(y_true, y_pred):
    return np.mean(abs(-y_true+y_pred))

In [35]:
tePredict = testPredict.reshape(-1)
testY = teY.reshape(-1)
print('Error Test Score > MSE == ', npMSE(testY, tePredict), ' MAE == ', npMAE(testY, tePredict), ' MAEMD == ', npMAEMD(testY, tePredict))

Error Test Score > MSE ==  0.015372308671441997  MAE ==  0.09759434327142198  MAEMD ==  0.1141773134758004


## Without Shuffling

In [39]:
input_shape = (None, 168, 37)  # Example input shape

with tf.device('/gpu:0'):
    model2 = PatchMixerModel(
        input_shape=input_shape,
        patch_size=8,
        stride=5,
        d_model=33,
        depth=4,
        kernel_size=8,
        forecasting=24,
        head_dropout=0.1)
    model2.build(input_shape=(168, 168, 37))
    model2.summary()

Model: "patch_mixer_model_3"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
patch_mixer_layer_12 (PatchM multiple                  1683      
_________________________________________________________________
patch_mixer_layer_13 (PatchM multiple                  1683      
_________________________________________________________________
patch_mixer_layer_14 (PatchM multiple                  1683      
_________________________________________________________________
patch_mixer_layer_15 (PatchM multiple                  1683      
_________________________________________________________________
dense_12 (Dense)             multiple                  297       
_________________________________________________________________
sequential_38 (Sequential)   (168, 24)                 798360    
_________________________________________________________________
sequential_39 (Sequential)   (168, 24)         

In [40]:
gc.collect()

6246

In [48]:
    model2.compile(loss=MAEMD, optimizer='adam', metrics=['mse','mae', MAEMD])
    early_stopping2 =EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)
    batch_size = 168
    epochs = 1000
    history2 = LossHistory()
    history2.init()

In [49]:
    %%time
    hist2 = model2.fit(trX, trY, epochs=epochs, batch_size=b_size, shuffle=False, validation_data=(vaX, vaY), callbacks=[history2, early_stopping2])  # , checkpoint

Epoch 1/1000
131/131 [==============================] - 24s 167ms/step - loss: 0.1452 - mse: 0.0314 - mae: 0.1342 - MAEMD: 0.1451 - val_loss: 0.1211 - val_mse: 0.0167 - val_mae: 0.1014 - val_MAEMD: 0.1225
Epoch 2/1000
131/131 [==============================] - 21s 164ms/step - loss: 0.1452 - mse: 0.0315 - mae: 0.1341 - MAEMD: 0.1452 - val_loss: 21.4141 - val_mse: 903.2235 - val_mae: 23.6605 - val_MAEMD: 21.6723
Epoch 3/1000
131/131 [==============================] - 21s 153ms/step - loss: 0.1439 - mse: 0.0326 - mae: 0.1335 - MAEMD: 0.1438 - val_loss: 2.6596 - val_mse: 14.7181 - val_mae: 2.8725 - val_MAEMD: 2.7123
Epoch 4/1000
131/131 [==============================] - 21s 164ms/step - loss: 0.1423 - mse: 0.0305 - mae: 0.1324 - MAEMD: 0.1422 - val_loss: 0.1235 - val_mse: 0.0182 - val_mae: 0.1061 - val_MAEMD: 0.1250
Epoch 5/1000
131/131 [==============================] - 21s 164ms/step - loss: 0.1413 - mse: 0.0305 - mae: 0.1323 - MAEMD: 0.1412 - val_loss: 0.1222 - val_mse: 0.0178 - val_m

In [43]:
gc.collect()

714

In [44]:
testPredict2 = model2.predict(teX, batch_size=b_size)

In [45]:
tePredict2 = testPredict2.reshape(-1)
testY = teY.reshape(-1)
print('Error Test Score > MSE == ', npMSE(testY, tePredict), ' MAE == ', npMAE(testY, tePredict), ' MAEMD == ', npMAEMD(testY, tePredict))
print('Error Test Score > MSE == ', npMSE(testY, tePredict2), ' MAE == ', npMAE(testY, tePredict2), ' MAEMD == ', npMAEMD(testY, tePredict2))

Error Test Score > MSE ==  0.015372308671441997  MAE ==  0.09759434327142198  MAEMD ==  0.1141773134758004
Error Test Score > MSE ==  0.01731172370530455  MAE ==  0.104071741798158  MAEMD ==  0.12051458036421082


In [46]:
from statsmodels.stats.diagnostic import acorr_ljungbox
from scipy.stats import norm

def diebold_mariano_test(forecast1, forecast2, actual, opt): 
    
    if opt==0: # MSE
        e1 = actual-forecast1
        e2 = actual-forecast2
        d = e1**2 - e2**2
    elif opt==1: # MAE
        e1 = abs(actual-forecast1)
        e2 = abs(actual-forecast2)
        d = e1 - e2
    else:
        e1 = np.multiply(abs(actual - forecast1), np.square(actual-np.mean(actual)))
        e2 = np.multiply(abs(actual - forecast2), np.square(actual-np.mean(actual)))
        d = e1-e2
    
    # Mean of the loss differentials
    mean_d = np.mean(d)
    
    # Standard deviation of the loss differentials
    std_d = np.std(d, ddof=1)
    
    # Calculate the test statistic
    test_stat = (mean_d / std_d) * np.sqrt(len(d))
    
    # Calculate the p-value using a two-tailed test
    p_value = 2 * (1 - norm.cdf(abs(test_stat)))
    
    return test_stat, p_value

In [47]:
diebold_mariano_test(testPredict, testPredict2, teY, 0), diebold_mariano_test(testPredict, testPredict2, teY, 1), diebold_mariano_test(testPredict, testPredict2, teY, 2)

((-6.022194600501482, 1.7206782665368792e-09),
 (-5.873473391046085, 4.267575626215603e-09),
 (-2.876314077657889, 0.004023491454638872))

In [27]:
np.savetxt('PatchMixer_prop', testPredict)
np.savetxt('PatchMixer_woshuffling', testPredict2)